In [1]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [3]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

# temp = AssistantAgent("aa", model_client=model, system_message="")

clarity_agent = AssistantAgent(
    name = "ClarityAgent",
    model_client=model,
    system_message= """
                    You are an expert editor focused on clarity and simplicity.
                    Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
                    Don't worry about persuasion or tone — just make the message easy to read and understand.
                    """
)

tone_agent  = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message= """
                    You are a communication coach focused on emotional tone and professionalism.
                    Your job is to make the email sound warm, confident, and human — while staying professional
                    and appropriate for the audience. Improve the emotional resonance, polish the phrasing,
                    and adjust any words that may come off as stiff, cold, or overly casual.
                    """
)

persuasion_agent  = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message= """
                    You are a persuasion expert trained in marketing, behavioral psychology, and copywriting.
                    Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. 
                    Remove weak or passive language.
                    """
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message= """
                    You are an advanced email-writing specialist. Your role is to read all
                    prior agent responses and revisions, and then **synthesize the best ideas** into a unified,
                    polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements;
                    Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional,
                    effective, and readable.
                    """
)

critic_agent  = AssistantAgent(
    "criticAgent",
    model_client=model,
    system_message= """
                    You are an email quality evaluator. Your job is to perform a final review
                    of the synthesized email and determine if it meets professional standards. Review the email for:
                    Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
                    Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone,
                    or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.
                    """
)

In [ ]:
text_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(max_messages=30)

termination_conditions = text_termination | max_message_termination

team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent
    ],
    termination_condition=termination_conditions
)

await Console(
    team.run_stream(task="안녕! 나 배고파 점심사줘~")
)

---------- TextMessage (user) ----------
Hi! Im hungry, please buy me lunch!
---------- TextMessage (ClarityAgent) ----------
Hi! I'm hungry. Please buy me lunch.
---------- TextMessage (ToneAgent) ----------
Subject: Lunch Request

Hi [Recipient's Name],

I hope this message finds you well! I’m feeling a bit hungry today and was wondering if you could treat me to lunch. I’d really appreciate it!

Thank you so much!

Best,  
[Your Name]
---------- TextMessage (PersuasionAgent) ----------
Subject: A Quick Favor - Lunch This Time?

Hi [Recipient's Name],

I hope you're having a fantastic day! I'm reaching out because I'm feeling a bit hungry and would love to share a delicious lunch together. It would be a wonderful treat, and I promise to return the favor soon!

How about we pick a spot you enjoy? I'm excited to spend some quality time with you and indulge in some great food.

Thank you in advance for considering it! I truly appreciate your generosity.

Looking forward to your reply!

B

TaskResult(messages=[TextMessage(id='0fe00df6-bba2-4e80-80cc-a20a56d063b3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 17, 675418, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, please buy me lunch!', type='TextMessage'), TextMessage(id='6ac78504-2037-424d-9cfa-5cd0fb076cb5', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=72, completion_tokens=10), metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 19, 67509, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry. Please buy me lunch.", type='TextMessage'), TextMessage(id='60c179d9-c297-4c24-8ab3-721d77baee36', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=110, completion_tokens=55), metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 21, 774669, tzinfo=datetime.timezone.utc), content="Subject: Lunch Request\n\nHi [Recipient's Name],\n\nI hope this message finds you well! I’m feeling a bit hungry today and was wondering if yo